In [1]:
import pandas as pd
import os
import kaleido
import plotly.graph_objects as go
import plotly.express as px

In [2]:
path = "data/consolidated/consolidado_alimentos.xlsx"
df = pd.read_excel(path)

df["fecha_scraping"] = pd.to_datetime(df["fecha_scraping"])


In [3]:
top10_subclases = (
    df.groupby("subclase_inei")["precio_por_unidad_estandar"]
    .count()
    .sort_values(ascending=False)
    .head(10)
    .index
)

# ===============================
# 2. Filtrar solo esas categorías
# ===============================
df_filtrado = df[df["subclase_inei"].isin(top10_subclases)]

# ===============================
# 3. Calcular precio promedio por categoría y fecha
# ===============================
df_line = (
    df_filtrado.groupby(["fecha_scraping", "subclase_inei"])["precio_por_unidad_estandar"]
    .mean()
    .reset_index()
)



# ===============================
# 5. Gráfico de líneas
# ===============================
fig = px.line(
    df_line,
    x="fecha_scraping",
    y="precio_por_unidad_estandar",
    color="subclase_inei",
    markers=True,
    template="plotly_white"
)

fig.update_layout(
    title=None,#"Evolución de Precios por Categoría (Top 10 por número de observaciones)",
    xaxis_title="Fecha",
    yaxis_title="Precio por unidad estandarizada",
    legend_title="Categoría",
    width=900,
    height=500
)



fig.show()

fig.write_image("results/Alimentos/evol_precios_alimentos.pdf")


In [4]:
#Seleccionar top 10 subclases con más observaciones
# ===============================
top10_subclases = (
    df.groupby("subclase_inei")["precio_por_unidad_estandar"]
    .count()
    .sort_values(ascending=False)
    .head(10)
    .index
)


fig = go.Figure()

for cat in top10_subclases:
    fig.add_trace(go.Box(
        y=df[df["subclase_inei"] == cat]["precio_por_unidad_estandar"],
        name=cat
    ))

fig.update_layout(
    title=None,
    yaxis_title="Precio por unidad estandarizada",
    template="plotly_white",
    showlegend=False,
    width=900,
    height=500
)

fig.show()


fig.write_image("results/Alimentos/grafico_boxplot_subclase_top10.pdf")

In [5]:
# ================================
# 3. Heatmap: TOP 10 Categorías vs Tienda
# ================================

# Tabla pivot original
tabla = pd.pivot_table(
    df,
    index="subclase_inei",
    columns="tienda",
    values="sku",
    aggfunc="count"
)

# ---- 1) Ordenar categorías por total de SKUs
tabla["TOTAL"] = tabla.sum(axis=1)

# ---- 2) Tomar solo las TOP 10 categorías
tabla_top10 = tabla.sort_values("TOTAL", ascending=False).head(10)

# Eliminar la columna TOTAL para el gráfico
tabla_top10 = tabla_top10.drop(columns="TOTAL")

# ---- 3) Generar el heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=tabla_top10.values,
        x=tabla_top10.columns,
        y=tabla_top10.index,
        colorscale="Viridis"
    )
)

fig.update_layout(
    title="",
    xaxis_title="Tienda",
    yaxis_title="Categoría (Top 10 por cantidad de SKUs)",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/Alimentos/skus_tiendas_alimentos.pdf")

In [6]:
df_vol = df.dropna(subset=["precio_por_unidad_estandar"])

# ==================================
# 2. Calcular métricas por categoría × tienda
# ==================================
vol_table = df_vol.groupby(["subclase_inei", "tienda"]).agg(
    precio_promedio=("precio_por_unidad_estandar", "mean"),
    precio_std=("precio_por_unidad_estandar", "std"),
    n_obs=("precio_por_unidad_estandar", "count")
).reset_index()

# Coeficiente de variación (CV)
vol_table["coef_variacion"] = vol_table["precio_std"] / vol_table["precio_promedio"]

# ==================================
# 3. Filtrar subclases con datos en TODAS las tiendas
# ==================================

# Lista de tiendas disponibles
tiendas = df_vol["tienda"].unique()

# Cuántas tiendas tienen datos por subclase
conteo_tiendas = vol_table.groupby("subclase_inei")["tienda"].nunique()

# Subclases válidas = aparecen en TODAS las tiendas
subclases_validas = conteo_tiendas[conteo_tiendas == len(tiendas)].index

# Filtramos solo subclases completas
vol_filtrado = vol_table[vol_table["subclase_inei"].isin(subclases_validas)]

# ==================================
# 4. Seleccionar TOP 10 subclases más volátiles
# ==================================
top10 = (
    vol_filtrado.groupby("subclase_inei")["coef_variacion"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
    .index
)

# Tabla final
vol_final = vol_filtrado[vol_filtrado["subclase_inei"].isin(top10)]

# ==================================
# 5. Gráfico (Plotly Go)
# ==================================
fig = go.Figure()

for tienda in vol_final["tienda"].unique():
    temp = vol_final[vol_final["tienda"] == tienda]
    
    fig.add_trace(go.Bar(
        x=temp["subclase_inei"],
        y=temp["coef_variacion"],
        name=tienda
    ))
#Volatilidad de Precios por Categoría y Tienda (TOP 10, CV alto y datos completos)
fig.update_layout(
    title=None,
    xaxis_title="Categoría",
    yaxis_title="Coeficiente de Variación (CV)",
    barmode="group",
    template="plotly_white",
    xaxis_tickangle=-45,
    width=900,     
    height=500 
)

fig.show()
fig.write_image("results/Alimentos/vol_categ_tienda.pdf")

In [7]:
# ================================
# 5. Ranking Top 20 Subclases Más Caras
# ================================

precio_promedio = (
    df.groupby("subclase_inei")["precio_por_unidad_estandar"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=precio_promedio.values,
    y=precio_promedio.index,
    orientation="h"
))

fig.update_layout(
    title="",
    xaxis_title="Precio promedio por unidad estandarizada",
    yaxis_title="Subclase",
    template="plotly_white",
    width=900,
    height=500
)

fig.show()
fig.write_image("results/Alimentos/mean_precio_subclase_alimentos.pdf")

In [8]:
# ===========================================================
#   RESÚMENES ESTADÍSTICOS DEL SECTOR ALIMENTOS
# ===========================================================

print("\n\n============================")
print(" RESUMEN 1: Evolución por subclase (df_line)")
print("============================\n")

for cat in df_line["subclase_inei"].unique():
    serie = df_line[df_line["subclase_inei"] == cat].set_index("fecha_scraping")["precio_por_unidad_estandar"]

    print(f"\n--- {cat.upper()} ---")
    print(f"Valores:\n{serie.to_string()}\n")
    print(f"Valor inicial: {serie.iloc[0]:.4f}")
    print(f"Valor final:   {serie.iloc[-1]:.4f}")
    print(f"Variación total: {((serie.iloc[-1]/serie.iloc[0])-1)*100:+.2f}%")
    print(f"Promedio: {serie.mean():.4f}")
    print(f"Desviación estándar: {serie.std():.4f}")
    print(f"Mínimo: {serie.min():.4f}")
    print(f"Máximo: {serie.max():.4f}")
    print("-------------------------")


print("\n\n============================")
print(" RESUMEN 2: Boxplot (TOP 10 subclases)")
print("============================\n")

for cat in top10_subclases:
    serie = df[df["subclase_inei"] == cat]["precio_por_unidad_estandar"]

    print(f"\n--- {cat.upper()} ---")
    print(f"Count: {len(serie)}")
    print(f"Promedio: {serie.mean():.4f}")
    print(f"Desviación estándar: {serie.std():.4f}")
    print(f"Mínimo: {serie.min():.4f}")
    print(f"Máximo: {serie.max():.4f}")
    print("-------------------------")


print("\n\n============================")
print(" RESUMEN 3: Heatmap SKUs por tienda (tabla_top10)")
print("============================\n")

print(tabla_top10)
print("\nTotales por subclase:")
print(tabla_top10.sum(axis=1))


print("\n\n============================")
print(" RESUMEN 4: Métricas por categoría × tienda (vol_table)")
print("============================\n")

for _, row in vol_table.iterrows():
    print(
        f"{row['subclase_inei']} ({row['tienda']}): "
        f"media={row['precio_promedio']:.4f}, "
        f"std={row['precio_std']:.4f}, "
        f"CV={row['coef_variacion']:.4f}, "
        f"obs={row['n_obs']}"
    )


print("\n\n============================")
print(" RESUMEN 5: TOP 10 subclases más caras")
print("============================\n")

print(precio_promedio)
print("\nDetalle:")

for cat, val in precio_promedio.items():
    serie = df[df["subclase_inei"] == cat]["precio_por_unidad_estandar"]
    print(f"{cat}: promedio={val:.4f}, min={serie.min():.4f}, max={serie.max():.4f}, n={len(serie)}")




 RESUMEN 1: Evolución por subclase (df_line)


--- AGUAS MINERALES Y BEBIDAS GASEOSAS ---
Valores:
fecha_scraping
2025-10-29    3.705712
2025-10-30    3.716734
2025-10-31    3.757384
2025-11-01    3.756990
2025-11-02    3.761793
2025-11-03    3.749820
2025-11-04    3.804332
2025-11-05    3.782727
2025-11-06    3.800677
2025-11-07    3.782258
2025-11-08    3.790077
2025-11-09    3.794273
2025-11-10    3.761939
2025-11-11    3.772869
2025-11-12    3.788703
2025-11-13    3.812504
2025-11-14    3.817873
2025-11-15    3.813696
2025-11-16    3.814937
2025-11-17    3.803808
2025-11-18    3.801536
2025-11-19    3.802591

Valor inicial: 3.7057
Valor final:   3.8026
Variación total: +2.61%
Promedio: 3.7815
Desviación estándar: 0.0308
Mínimo: 3.7057
Máximo: 3.8179
-------------------------

--- CARNE DE RES FRESCA, REFRIGERADA Y CONGELADA ---
Valores:
fecha_scraping
2025-10-29    31.272101
2025-10-30    31.279845
2025-10-31    31.279334
2025-11-01    31.328905
2025-11-02    31.288503
2025-11-03